# **Reproduce DenseNet with Tensorflow and MNIST Dataset**
### **Nama : Raihan Azhar Rafi**
### **Kelas : TK-42-02**
### **Nim : 1103180166**

Sumber Paper Saya Peroleh Dari :https://arxiv.org/pdf/1608.06993.pdf

Sumber Pencarian Referensi Paper Projek Saya Peroleh Dari : https://paperswithcode.com/paper/densely-connected-convolutional-networks  

Sumber Reproduce Projek DenseNet Untuk Kebutuhan Tugas UAS Machine Learning
Saya Peroleh Dari : https://github.com/yw0nam/DenseNet

In [1]:
import tensorflow as tf
import numpy as np

# If you aren't using gpu, don't excecute below code
# You can read document for below code here
# https://www.tensorflow.org/guide/gpu

# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
# tf.config.experimental.set_memory_growth(gpus[1], True)

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape([x_train.shape[0], x_train.shape[1], x_train.shape[2], 1])
x_test = x_test.reshape([x_test.shape[0], x_test.shape[1], x_test.shape[2], 1])

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
#If you can't load mnist data form above code, Download mnist.npz and excecute this cell

# from modules.MnistData import MnistData
# data = MnistData('./mnist.npz')

# x_train, y_train = data.x_train, data.y_train
# x_test, y_test = data.x_test, data.y_test

# x_train = x_train.reshape([x_train.shape[0], x_train.shape[1], x_train.shape[2], 1])
# x_test = x_test.reshape([x_test.shape[0], x_test.shape[1], x_test.shape[2], 1])

In [4]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Dropout
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import Dense, Flatten, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Input
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D
import tensorflow as tf

In [5]:
def Build_conv2d(x, channel, stride):
    return (Conv2D(channel, kernel_size=(3, 3), padding='same',
                   kernel_initializer='he_uniform', use_bias=False,
                   strides=stride, activation=None))(x)

In [6]:
def Build_Dense_block(name, x, growthRate):
        shape = x.get_shape().as_list()
        input_channel = shape[3]
        with tf.compat.v1.variable_scope(name) as scope:
            dense = BatchNormalization()(x)
            dense = Activation('relu')
            dense = Build_conv2d(x, growthRate, (1, 1))
            return tf.concat([x, dense], 3)

In [7]:
def Build_transition(name, x):
    shape = x.get_shape().as_list()
    input_channel = shape[3]
    with tf.compat.v1.variable_scope(name) as scope:
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(input_channel, kernel_size=(1, 1),
                   activation=None, strides=(1, 1),
                   use_bias=False)(x)
        return AveragePooling2D([2, 2])(x)

In [8]:
def Build_net(shape, dense_block_num, growthRate):

    channel = 32
    row, col = shape[0], shape[1]
    input_layer = Input(shape=(row, col, 1), name='input')

    x = Build_conv2d(input_layer, channel, (1, 1))

    name_1, name_2, name_3 = 'dense_block_1', 'dense_block_2', 'dense_block_3'

    """
    ====================================
    =========Build First Block==========
    ====================================
    """

    with tf.name_scope(name_1) as scope:

        for i in range(dense_block_num):
            x = Build_Dense_block('dense_layer_{num}'.format(num=i), x, growthRate)

        x = Build_transition('transtion_1', x)

    """
    ====================================
    =========Build Second Block=========
    ====================================
    """

    with tf.name_scope(name_2) as scope:

        for i in range(dense_block_num):
            x = Build_Dense_block('dense_layer_{num}'.format(num=i), x, growthRate)

        x = Build_transition('transtion_2', x)

    """
    ====================================
    =========Build Third Block==========
    ====================================
    """

    with tf.name_scope(name_3) as scope:

        for i in range(dense_block_num):
            x = Build_Dense_block('dense_layer_{num}'.format(num=i), x, growthRate)

        x = Build_transition('transtion_3', x)

    x = BatchNormalization(name='bn_last')(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    out = Dense(10, activation='softmax')(x)
    model = Model(input_layer, out)

    return model

In [9]:
model = Build_net([28, 28], 3, 32)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 28, 28, 32)   288         ['input[0][0]']                  
                                                                                                  
 conv2d_1 (Conv2D)              (None, 28, 28, 32)   9216        ['conv2d[0][0]']                 
                                                                                                  
 tf.concat (TFOpLambda)         (None, 28, 28, 64)   0           ['conv2d[0][0]',                 
                                                                  'conv2d_1[0][0]']           

In [11]:
epoch = 10
batch_size = 100

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
train_history = model.fit(x_train, y_train, epochs=epoch, batch_size=batch_size,  verbose=1,
                          validation_split=0.1 )

Epoch 1/10
540/540 [==============================] - 1410s 3s/step - loss: 0.1520 - accuracy: 0.9612 - val_loss: 0.0937 - val_accuracy: 0.9715
Epoch 2/10
540/540 [==============================] - 1410s 3s/step - loss: 0.0458 - accuracy: 0.9864 - val_loss: 0.2975 - val_accuracy: 0.9018
Epoch 3/10
540/540 [==============================] - 1422s 3s/step - loss: 0.0357 - accuracy: 0.9888 - val_loss: 0.0339 - val_accuracy: 0.9898
Epoch 4/10
540/540 [==============================] - 1411s 3s/step - loss: 0.0293 - accuracy: 0.9910 - val_loss: 0.0661 - val_accuracy: 0.9825
Epoch 5/10
540/540 [==============================] - 1420s 3s/step - loss: 0.0245 - accuracy: 0.9920 - val_loss: 0.0318 - val_accuracy: 0.9915
Epoch 6/10
540/540 [==============================] - 1420s 3s/step - loss: 0.0261 - accuracy: 0.9917 - val_loss: 0.1321 - val_accuracy: 0.9573
Epoch 7/10
540/540 [==============================] - 1427s 3s/step - loss: 0.0202 - accuracy: 0.9936 - val_loss: 0.0435 - val_accuracy:

In [13]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 53s - loss: 0.0451 - accuracy: 0.9858 - 53s/epoch - 170ms/step


[0.04512866958975792, 0.98580002784729]